In [1]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving resonant-rock-448902-j3-a8e32e697012.json to resonant-rock-448902-j3-a8e32e697012.json


In [3]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")


Authentication successful!


In [4]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1DwsxqXhuJBdI0Y8320g0h87hMe61tDx6khpJaG36lJs


In [5]:
# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'email@example.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document shared with ankitdani1997@gmail.com


In [6]:
# Step 6: Upload and Read the Markdown File
print("Please upload your .md file containing the meeting notes.")
uploaded_md = files.upload()
md_file_name = list(uploaded_md.keys())[0]  # Get the uploaded .md file name

# Read the content of the uploaded file
with open(md_file_name, 'r') as file:
    meeting_notes = file.read()

print("Markdown file uploaded and read successfully!")


Please upload your .md file containing the meeting notes.


Saving meeting_notes.md to meeting_notes.md
Markdown file uploaded and read successfully!


In [9]:
# Step 7
def add_text_with_style(doc_id, text, style=None, bold=False, underline=False, color=None):
    """
    Adds text to the Google Doc with optional paragraph and text styles.
    """
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Insert text
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        }
    ]

    # Apply paragraph styles if provided
    if style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        })

    # Apply text styles (bold, underline, color)
    text_style = {}
    if bold:
        text_style['bold'] = True
    if underline:
        text_style['underline'] = True
    if color:
        text_style['foregroundColor'] = {
            'color': {
                'rgbColor': {
                    'red': color[0],
                    'green': color[1],
                    'blue': color[2]
                }
            }
        }

    if text_style:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'textStyle': text_style,
                'fields': ','.join(text_style.keys())
            }
        })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def add_checkbox(doc_id, text):
    """
    Adds a functional checkbox to the Google Doc.
    """
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def parse_markdown_and_add_content(doc_id, markdown):
    """
    Parses markdown and adds styled content to the Google Doc.
    """
    lines = markdown.split('\n')
    in_footer = False  # Track if we are in the footer section
    for line in lines:
        line = line.strip()
        if line.startswith('---'):  # Footer starts after ---
            in_footer = True
            continue

        if in_footer:  # Footer content
            add_text_with_style(doc_id, line, bold=True)
        elif line.startswith('# '):  # Heading 1
            add_text_with_style(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_with_style(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif '@' in line:
            # Improved @ mention handling
            parts = []
            start = 0
            while '@' in line[start:]:
                at_index = line.find('@', start)

                # Add text before @ with default styling
                if at_index > start:
                    parts.append((line[start:at_index], False))

                # Find end of mention (space or end of line)
                space_index = line.find(' ', at_index)
                if space_index == -1:
                    space_index = len(line)

                # Add mention with special styling
                parts.append((line[at_index:space_index], True))

                # Update start for next iteration
                start = space_index

            # Add any remaining text
            if start < len(line):
                parts.append((line[start:], False))

            # Add parts with appropriate styling
            for text, is_mention in parts:
                if is_mention:
                    add_text_with_style(
                        doc_id,
                        text,
                        underline=False,
                        color=(0.0, 0.0, 1.0)  # Blue color for @mentions
                    )
                else:
                    add_text_with_style(doc_id, text)
        elif line.startswith('-'):  # Bullet point
            add_text_with_style(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_with_style(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_with_style(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")

KeyboardInterrupt: 